<a href="https://colab.research.google.com/github/LiamSwick/Data-Science-Sandbox/blob/main/Absenteeism_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [427]:
import numpy as np
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [428]:
load = pd.read_csv('/content/drive/MyDrive/Absenteeism_preprocessed.csv')
load = load.drop(['Date'], axis = 1)
load.head()

,Reason 1,Reason 2,Reason 3,Reason 4,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours,Months,Day of the Week
0,0,0,0,1,289,36,33,239.554,30,0,2,1,4,7,1
1,0,0,0,0,118,13,50,239.554,31,0,1,0,0,7,1
2,0,0,0,1,179,51,38,239.554,31,0,0,0,2,7,2
3,1,0,0,1,279,5,39,239.554,24,0,2,0,4,7,3
4,0,0,0,1,289,36,33,239.554,30,0,2,1,2,7,3


In [429]:
df = load.copy()
df.head()

,Reason 1,Reason 2,Reason 3,Reason 4,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours,Months,Day of the Week
0,0,0,0,1,289,36,33,239.554,30,0,2,1,4,7,1
1,0,0,0,0,118,13,50,239.554,31,0,1,0,0,7,1
2,0,0,0,1,179,51,38,239.554,31,0,0,0,2,7,2
3,1,0,0,1,279,5,39,239.554,24,0,2,0,4,7,3
4,0,0,0,1,289,36,33,239.554,30,0,2,1,2,7,3


In [430]:
absent_med = df['Absenteeism Time in Hours'].median()

In [431]:
targets = np.where(df['Absenteeism Time in Hours'] > absent_med, 1, 0)

In [432]:
df["Excessive Absenteeism"] = targets
df.head()

,Reason 1,Reason 2,Reason 3,Reason 4,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours,Months,Day of the Week,Excessive Absenteeism
0,0,0,0,1,289,36,33,239.554,30,0,2,1,4,7,1,1
1,0,0,0,0,118,13,50,239.554,31,0,1,0,0,7,1,0
2,0,0,0,1,179,51,38,239.554,31,0,0,0,2,7,2,0
3,1,0,0,1,279,5,39,239.554,24,0,2,0,4,7,3,1
4,0,0,0,1,289,36,33,239.554,30,0,2,1,2,7,3,0


In [433]:
print(f"The target class represents {targets.sum()/targets.shape[0] *100: .2f}% of the data")

The target class represents  45.57% of the data


In [434]:
df_w_targets = df.drop(['Absenteeism Time in Hours', 'Day of the Week', 'Distance to Work', 'Daily Work Load Average'], axis = 1)
df_w_targets.head()

,Reason 1,Reason 2,Reason 3,Reason 4,Transportation Expense,Age,Body Mass Index,Education,Children,Pets,Months,Excessive Absenteeism
0,0,0,0,1,289,33,30,0,2,1,7,1
1,0,0,0,0,118,50,31,0,1,0,7,0
2,0,0,0,1,179,38,31,0,0,0,7,0
3,1,0,0,1,279,39,24,0,2,0,7,1
4,0,0,0,1,289,33,30,0,2,1,7,0


In [435]:
unscaled_inputs = df_w_targets.iloc[:,:-1]


In [436]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler

class CustomScaler(BaseEstimator, TransformerMixin):

    def __init__(self, columns):
        # Explicitly name the columns and initialize the internal scaler
        self.scaler = StandardScaler()
        self.columns = columns
        self.mean_ = None
        self.var_ = None

    def fit(self, X, y=None):
        # Fit only the specific columns we want to scale
        self.scaler.fit(X[self.columns], y)
        self.mean_ = np.mean(X[self.columns], axis=0)
        self.var_ = np.var(X[self.columns], axis=0)
        return self

    def transform(self, X, y=None, copy=None):
        init_col_order = X.columns
        # Transform only the target columns
        X_scaled = pd.DataFrame(self.scaler.transform(X[self.columns]), columns=self.columns)
        # Keep the rest (like the Reason dummies) unscaled
        X_not_scaled = X.loc[:, ~X.columns.isin(self.columns)].reset_index(drop=True)
        # Recombine and return in the original order
        return pd.concat([X_not_scaled, X_scaled], axis=1)[init_col_order]

In [437]:
unscaled_inputs.columns.values

array(['Reason 1', 'Reason 2', 'Reason 3', 'Reason 4',
       'Transportation Expense', 'Age', 'Body Mass Index', 'Education',
       'Children', 'Pets', 'Months'], dtype=object)

In [438]:
features_to_scale = ['Transportation Expense', 'Age', 'Body Mass Index',
                     'Children', 'Pets', 'Months']

In [439]:
scaler = CustomScaler(features_to_scale)

In [440]:
scaler.fit(unscaled_inputs)

CustomScaler(columns=['Transportation Expense', 'Age', 'Body Mass Index',
                      'Children', 'Pets', 'Months'])

In [441]:
scaled_inputs = scaler.transform(unscaled_inputs)
scaled_inputs.head()

,Reason 1,Reason 2,Reason 3,Reason 4,Transportation Expense,Age,Body Mass Index,Education,Children,Pets,Months
0,0,0,0,1,1.005844,-0.536062,0.767431,0,0.880469,0.268487,0.182726
1,0,0,0,0,-1.574681,2.130803,1.002633,0,-0.019280,-0.589690,0.182726
2,0,0,0,1,-0.654143,0.248310,1.002633,0,-0.919030,-0.589690,0.182726
3,1,0,0,1,0.854936,0.405184,-0.643782,0,0.880469,-0.589690,0.182726
4,0,0,0,1,1.005844,-0.536062,0.767431,0,0.880469,0.268487,0.182726


In [442]:
x_train, x_test, y_train, y_test = train_test_split(scaled_inputs, targets, train_size = 0.8, random_state = 21)

In [443]:
log_mod = LogisticRegression()

In [444]:
log_mod.fit(x_train, y_train)

LogisticRegression()

In [445]:
acc = log_mod.score(x_train, y_train)

In [446]:
from sklearn.metrics import f1_score, roc_auc_score

In [447]:
yhat = log_mod.predict(x_train)

In [448]:
f1 = f1_score(y_train, yhat)
print(f"Accuracy is {acc*100:.2f}% with an F1-score of {f1:0.2f}")

Accuracy is 77.14% with an F1-score of 0.75


In [449]:
intercept = log_mod.intercept_[0]
print(intercept)

-3.235539671406899


In [450]:
coefs = log_mod.coef_
print(coefs, len(coefs))

[[ 1.93998508  0.67488192  2.2921362   2.4334647   0.60060663 -0.19268878
   0.35425638  0.10723457  0.4868911  -0.27643623  0.088531  ]] 1


In [451]:
feature_name = unscaled_inputs.columns.values
print(feature_name)

['Reason 1' 'Reason 2' 'Reason 3' 'Reason 4' 'Transportation Expense'
 'Age' 'Body Mass Index' 'Education' 'Children' 'Pets' 'Months']


In [452]:
summary_table = pd.DataFrame(columns = ['Feature name'], data = feature_name)
summary_table['Coefficients (Log Odds)'] = np.transpose(coefs).round(2)
summary_table

,Feature name,Coefficients (Log Odds)
0,Reason 1,1.94
1,Reason 2,0.67
2,Reason 3,2.29
3,Reason 4,2.43
4,Transportation Expense,0.60
5,Age,-0.19
6,Body Mass Index,0.35
7,Education,0.11
8,Children,0.49
9,Pets,-0.28


In [453]:
from scipy.special import expit
summary_table.index += 1
summary_table.loc[0] = ["Intercept", intercept]
summary_table = summary_table.sort_index()

probabilities = expit(summary_table['Coefficients (Log Odds)'])
summary_table["Odds Ratio"] = np.exp(summary_table['Coefficients (Log Odds)']).round(2)
summary_table['Probabilities'] = probabilities.round(2)
summary_table

,Feature name,Coefficients (Log Odds),Odds Ratio,Probabilities
0,Intercept,-3.23554,0.04,0.04
1,Reason 1,1.94000,6.96,0.87
2,Reason 2,0.67000,1.95,0.66
3,Reason 3,2.29000,9.87,0.91
4,Reason 4,2.43000,11.36,0.92
5,Transportation Expense,0.60000,1.82,0.65
6,Age,-0.19000,0.83,0.45
7,Body Mass Index,0.35000,1.42,0.59
8,Education,0.11000,1.12,0.53
9,Children,0.49000,1.63,0.62


In [454]:
summary_table.sort_values("Odds Ratio", ascending=False)

,Feature name,Coefficients (Log Odds),Odds Ratio,Probabilities
4,Reason 4,2.43000,11.36,0.92
3,Reason 3,2.29000,9.87,0.91
1,Reason 1,1.94000,6.96,0.87
2,Reason 2,0.67000,1.95,0.66
5,Transportation Expense,0.60000,1.82,0.65
9,Children,0.49000,1.63,0.62
7,Body Mass Index,0.35000,1.42,0.59
8,Education,0.11000,1.12,0.53
11,Months,0.09000,1.09,0.52
6,Age,-0.19000,0.83,0.45


In [456]:
test_accuracy = log_mod.score(x_test, y_test)
print(f"Test accuracy is: {test_accuracy*100:0.2f}%")

Test accuracy is: 78.57%


In [459]:
predicted_proba = log_mod.predict_proba(x_test)

In [461]:
import pickle

In [462]:
with open('model', 'wb') as file:
    pickle.dump(log_mod, file)

In [463]:
with open('scaler', 'wb') as file:
    pickle.dump(scaler, file)